In [1]:
from fredapi import Fred
import pandas as pd
import numpy as np

# Initialize Fred with your API key
fred = Fred(api_key='db61e0d65c4d2a1053221aec21822d4e')  # Replace with your actual API key securely

# Define date range
start_date = '2002-01-01'
end_date = '2024-11-28'

# Define the indicators and their series IDs
indicators = {
    'Effective Federal Funds Rate': 'FEDFUNDS',
    '10-Year Treasury Rate': 'DGS10',
    'Consumer Price Index': 'CPIAUCSL',
    'Producer Price Index': 'PPIACO',
    'Unemployment Rate': 'UNRATE',
    'Nonfarm Payroll Employment': 'PAYEMS',
    'Real GDP': 'GDPC1',
    'Housing Starts': 'HOUST',
    'Industrial Production Index': 'INDPRO',
    'M2 Money Stock': 'M2SL',
    'Crude Oil Prices': 'DCOILWTICO',
    'Retail Sales': 'RSXFS',  # Corrected series ID for Retail Sales
    'Total Business Inventories': 'BUSINV'
}

# Fetch the data with date range
economic_data = pd.DataFrame()

for name, series_id in indicators.items():
    try:
        data = fred.get_series(
            series_id,
            observation_start=start_date,
            observation_end=end_date
        )
        if data is not None and not data.empty:
            economic_data[name] = data
            print(f"Successfully fetched data for {name}")
    except Exception as e:
        print(f"Error fetching {name}: {e}")

# Convert index to datetime if not already
economic_data.index = pd.to_datetime(economic_data.index)

Successfully fetched data for Effective Federal Funds Rate
Successfully fetched data for 10-Year Treasury Rate
Successfully fetched data for Consumer Price Index
Successfully fetched data for Producer Price Index
Successfully fetched data for Unemployment Rate
Successfully fetched data for Nonfarm Payroll Employment
Successfully fetched data for Real GDP
Successfully fetched data for Housing Starts
Successfully fetched data for Industrial Production Index
Successfully fetched data for M2 Money Stock
Successfully fetched data for Crude Oil Prices
Successfully fetched data for Retail Sales
Successfully fetched data for Total Business Inventories


In [2]:
economic_data.head()

,Effective Federal Funds Rate,10-Year Treasury Rate,Consumer Price Index,Producer Price Index,Unemployment Rate,Nonfarm Payroll Employment,Real GDP,Housing Starts,Industrial Production Index,M2 Money Stock,Crude Oil Prices,Retail Sales,Total Business Inventories
2002-01-01,1.73,NaN,177.7,128.5,5.7,130853.0,14372.785,1698.0,88.4634,5461.1,NaN,256307.0,1117388.0
2002-02-01,1.74,5.02,178.0,128.4,5.7,130732.0,NaN,1829.0,88.4578,5490.4,20.40,257670.0,1111859.0
2002-03-01,1.73,4.98,178.5,129.8,5.7,130720.0,NaN,1642.0,89.1265,5502.2,22.37,257059.0,1108193.0
2002-04-01,1.75,5.44,179.3,130.8,5.9,130616.0,14460.848,1592.0,89.5507,5502.3,26.82,261333.0,1106806.0
2002-05-01,1.75,5.08,179.5,130.8,5.8,130632.0,NaN,1764.0,89.9348,5528.5,26.58,257573.0,1110725.0


In [3]:
economic_data.shape

(274, 13)

In [4]:
economic_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 274 entries, 2002-01-01 to 2024-10-01
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Effective Federal Funds Rate  274 non-null    float64
 1   10-Year Treasury Rate         178 non-null    float64
 2   Consumer Price Index          274 non-null    float64
 3   Producer Price Index          274 non-null    float64
 4   Unemployment Rate             274 non-null    float64
 5   Nonfarm Payroll Employment    274 non-null    float64
 6   Real GDP                      91 non-null     float64
 7   Housing Starts                274 non-null    float64
 8   Industrial Production Index   274 non-null    float64
 9   M2 Money Stock                274 non-null    float64
 10  Crude Oil Prices              178 non-null    float64
 11  Retail Sales                  274 non-null    float64
 12  Total Business Inventories    273 non-null   

In [5]:

# Ensure the DataFrame is sorted by date
economic_data.sort_index(inplace=True)

# Fill missing values
def fill_missing_values(df):
    df_filled = df.copy()

    # Ensure index is DatetimeIndex
    if not isinstance(df_filled.index, pd.DatetimeIndex):
        df_filled.index = pd.to_datetime(df_filled.index)
    df_filled.sort_index(inplace=True)

    # Create Month and Year columns once
    df_filled['Month'] = df_filled.index.month
    df_filled['Year'] = df_filled.index.year

    # Process each column individually
    for column in df.columns:
        col_data = df_filled[['Year', 'Month', column]].copy()

        # Calculate monthly means
        monthly_means = col_data.groupby(['Year', 'Month'])[column].mean().rename('Monthly_Mean').reset_index()

        # Merge monthly means back into col_data
        col_data = col_data.merge(monthly_means, on=['Year', 'Month'], how='left')

        # Fill missing values with Monthly Mean
        null_mask = col_data[column].isnull()
        col_data.loc[null_mask, column] = col_data.loc[null_mask, 'Monthly_Mean']

        # Calculate yearly means
        yearly_means = col_data.groupby('Year')[column].mean().rename('Yearly_Mean').reset_index()

        # Merge yearly means into col_data
        col_data = col_data.merge(yearly_means, on='Year', how='left')

        # Fill remaining missing values with Yearly Mean
        still_null_mask = col_data[column].isnull()
        col_data.loc[still_null_mask, column] = col_data.loc[still_null_mask, 'Yearly_Mean']

        # Update the filled values back into df_filled
        df_filled.loc[:, column] = col_data[column].values

    # Drop the auxiliary columns
    df_filled.drop(['Month', 'Year'], axis=1, inplace=True)

    return df_filled


# Apply the function to fill missing values
economic_data_filled = fill_missing_values(economic_data)
if not isinstance(economic_data_filled.index, pd.DatetimeIndex):
    economic_data_filled.index = pd.to_datetime(economic_data_filled.index)
# Ensure the DataFrame is sorted by date
economic_data_filled.sort_index(inplace=True)
print(economic_data_filled.index)
# Preview the filled data
print(economic_data_filled.head())


DatetimeIndex(['2002-01-01', '2002-02-01', '2002-03-01', '2002-04-01',
               '2002-05-01', '2002-06-01', '2002-07-01', '2002-08-01',
               '2002-09-01', '2002-10-01',
               ...
               '2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01',
               '2024-05-01', '2024-06-01', '2024-07-01', '2024-08-01',
               '2024-09-01', '2024-10-01'],
              dtype='datetime64[ns]', length=274, freq=None)
            Effective Federal Funds Rate  10-Year Treasury Rate  \
2002-01-01                          1.73                4.69625   
2002-02-01                          1.74                5.02000   
2002-03-01                          1.73                4.98000   
2002-04-01                          1.75                5.44000   
2002-05-01                          1.75                5.08000   

            Consumer Price Index  Producer Price Index  Unemployment Rate  \
2002-01-01                 177.7                 128.5               

In [6]:
print(economic_data_filled.isna().sum())

Effective Federal Funds Rate    0
10-Year Treasury Rate           0
Consumer Price Index            0
Producer Price Index            0
Unemployment Rate               0
Nonfarm Payroll Employment      0
Real GDP                        0
Housing Starts                  0
Industrial Production Index     0
M2 Money Stock                  0
Crude Oil Prices                0
Retail Sales                    0
Total Business Inventories      0
dtype: int64


In [7]:
economic_data_filled

,Effective Federal Funds Rate,10-Year Treasury Rate,Consumer Price Index,Producer Price Index,Unemployment Rate,Nonfarm Payroll Employment,Real GDP,Housing Starts,Industrial Production Index,M2 Money Stock,Crude Oil Prices,Retail Sales,Total Business Inventories
2002-01-01,1.73,4.696250,177.700,128.500,5.7,130853.0,14372.785000,1698.0,88.4634,5461.1,25.9025,256307.0,1.117388e+06
2002-02-01,1.74,5.020000,178.000,128.400,5.7,130732.0,14472.711500,1829.0,88.4578,5490.4,20.4000,257670.0,1.111859e+06
2002-03-01,1.73,4.980000,178.500,129.800,5.7,130720.0,14472.711500,1642.0,89.1265,5502.2,22.3700,257059.0,1.108193e+06
2002-04-01,1.75,5.440000,179.300,130.800,5.9,130616.0,14460.848000,1592.0,89.5507,5502.3,26.8200,261333.0,1.106806e+06
2002-05-01,1.75,5.080000,179.500,130.800,5.8,130632.0,14472.711500,1764.0,89.9348,5528.5,26.5800,257573.0,1.110725e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-01,5.33,4.175714,313.049,255.914,4.1,158548.0,23221.394667,1329.0,103.2768,21020.1,79.0500,607986.0,2.565930e+06
2024-07-01,5.33,4.480000,313.534,257.326,4.3,158692.0,23386.733000,1262.0,102.5381,21039.4,84.7000,615960.0,2.574892e+06
2024-08-01,5.33,3.990000,314.121,255.394,4.2,158770.0,23221.394667,1379.0,103.0449,21141.3,77.7400,614535.0,2.583708e+06
2024-09-01,5.13,4.175714,314.686,252.737,4.1,158993.0,23221.394667,1353.0,102.5497,21222.7,79.0500,619411.0,2.587145e+06


In [8]:
economic_data_filled.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 274 entries, 2002-01-01 to 2024-10-01
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Effective Federal Funds Rate  274 non-null    float64
 1   10-Year Treasury Rate         274 non-null    float64
 2   Consumer Price Index          274 non-null    float64
 3   Producer Price Index          274 non-null    float64
 4   Unemployment Rate             274 non-null    float64
 5   Nonfarm Payroll Employment    274 non-null    float64
 6   Real GDP                      274 non-null    float64
 7   Housing Starts                274 non-null    float64
 8   Industrial Production Index   274 non-null    float64
 9   M2 Money Stock                274 non-null    float64
 10  Crude Oil Prices              274 non-null    float64
 11  Retail Sales                  274 non-null    float64
 12  Total Business Inventories    274 non-null   

# Stock Data

In [9]:
stock_data = pd.read_parquet('sp500_50stocks_data.parquet')
# Convert index to DatetimeIndex if not already
if not isinstance(stock_data.index, pd.DatetimeIndex):
    stock_data.index = pd.to_datetime(stock_data.index)

# Sort by date
stock_data.sort_index(inplace=True)
stock_data

Ticker             AMD                                                  \
Price             Open        High         Low       Close   Adj Close   
Date                                                                     
2003-01-02    6.560000    7.110000    6.470000    7.010000    7.010000   
2003-01-03    7.010000    7.190000    6.860000    6.940000    6.940000   
2003-01-06    7.050000    7.230000    7.000000    7.160000    7.160000   
2003-01-07    7.250000    7.480000    7.100000    7.170000    7.170000   
2003-01-08    7.060000    7.100000    6.630000    6.690000    6.690000   
...                ...         ...         ...         ...         ...   
2024-11-20  138.960007  140.770004  135.479996  137.600006  137.600006   
2024-11-21  138.869995  140.279999  134.929993  137.490005  137.490005   
2024-11-22  137.350006  139.130005  137.039993  138.350006  138.350006   
2024-11-25  140.490005  142.350006  139.050003  141.130005  141.130005   
2024-11-26  142.550003  142.800003  136.619995  137.720001  137.720001   

Ticker                      TSLA                                      ...  \
Price         Volume        Open        High         Low       Close  ...   
Date                                                                  ...   
2003-01-02  17474800         NaN         NaN         NaN         NaN  ...   
2003-01-03   6235700         NaN         NaN         NaN         NaN  ...   
2003-01-06   6833200         NaN         NaN         NaN         NaN  ...   
2003-01-07   9785100         NaN         NaN         NaN         NaN  ...   
2003-01-08  13952300         NaN         NaN         NaN         NaN  ...   
...              ...         ...         ...         ...         ...  ...   
2024-11-20  28843100  345.000000  346.600006  334.299988  342.029999  ...   
2024-11-21  29311400  343.809998  347.989990  335.279999  339.640015  ...   
2024-11-22  21784700  341.089996  361.529999  337.700012  352.559998  ...   
2024-11-25  30923100  360.140015  361.929993  338.200012  338.589996  ...   
2024-11-26  32092400  341.000000  346.959991  335.660004  338.230011  ...   

Ticker             VZ                                        AAPL              \
Price             Low      Close  Adj Close    Volume        Open        High   
Date                                                                            
2003-01-02  35.136539  36.125793  12.283099   7294044    0.256429    0.266429   
2003-01-03  35.847004  36.377605  12.368718   5745323    0.264286    0.266607   
2003-01-06  37.034111  39.633156  13.475639  20465734    0.268393    0.274643   
2003-01-07  38.850742  39.228458  13.338038  14728528    0.264107    0.267857   
2003-01-08  36.602436  36.791294  12.620769  18176011    0.260357    0.262679   
...               ...        ...        ...       ...         ...         ...   
2024-11-20  41.750000  42.220001  42.220001  14706500  228.059998  229.929993   
2024-11-21  42.009998  42.500000  42.500000  12534100  228.880005  230.160004   
2024-11-22  42.340000  43.150002  43.150002  17017400  228.059998  230.720001   
2024-11-25  43.310001  43.980000  43.980000  28365900  231.460007  233.250000   
2024-11-26  43.750000  44.369999  44.369999  17247000  233.330002  235.570007   

Ticker                                                     
Price              Low       Close   Adj Close     Volume  
Date                                                       
2003-01-02    0.256250    0.264286    0.222938  181428800  
2003-01-03    0.260536    0.266071    0.224443  147453600  
2003-01-06    0.265714    0.266071    0.224443  390532800  
2003-01-07    0.258393    0.265179    0.223691  342344800  
2003-01-08    0.257857    0.259821    0.219171  229644800  
...                ...         ...         ...        ...  
2024-11-20  225.889999  229.000000  229.000000   35169600  
2024-11-21  225.710007  228.520004  228.520004   42108300  
2024-11-22  228.059998  229.869995  229.869995   38168300  
2024-11-25  229.740005  232.8

In [10]:
print(f"Economic data date range: {economic_data_filled.index.min()} to {economic_data_filled.index.max()}")
print(f"Stock data date range: {stock_data.index.min()} to {stock_data.index.max()}")

Economic data date range: 2002-01-01 00:00:00 to 2024-10-01 00:00:00
Stock data date range: 2003-01-02 00:00:00 to 2024-11-26 00:00:00


In [11]:
stock_data.columns

MultiIndex([( 'AMD',      'Open'),
            ( 'AMD',      'High'),
            ( 'AMD',       'Low'),
            ( 'AMD',     'Close'),
            ( 'AMD', 'Adj Close'),
            ( 'AMD',    'Volume'),
            ('TSLA',      'Open'),
            ('TSLA',      'High'),
            ('TSLA',       'Low'),
            ('TSLA',     'Close'),
            ...
            (  'VZ',       'Low'),
            (  'VZ',     'Close'),
            (  'VZ', 'Adj Close'),
            (  'VZ',    'Volume'),
            ('AAPL',      'Open'),
            ('AAPL',      'High'),
            ('AAPL',       'Low'),
            ('AAPL',     'Close'),
            ('AAPL', 'Adj Close'),
            ('AAPL',    'Volume')],
           names=['Ticker', 'Price'], length=300)

In [12]:
isinstance(stock_data.columns, pd.MultiIndex)

True

In [13]:
stock_data.index

DatetimeIndex(['2003-01-02', '2003-01-03', '2003-01-06', '2003-01-07',
               '2003-01-08', '2003-01-09', '2003-01-10', '2003-01-13',
               '2003-01-14', '2003-01-15',
               ...
               '2024-11-13', '2024-11-14', '2024-11-15', '2024-11-18',
               '2024-11-19', '2024-11-20', '2024-11-21', '2024-11-22',
               '2024-11-25', '2024-11-26'],
              dtype='datetime64[ns]', name='Date', length=5514, freq=None)

In [14]:
economic_data_filled.index

DatetimeIndex(['2002-01-01', '2002-02-01', '2002-03-01', '2002-04-01',
               '2002-05-01', '2002-06-01', '2002-07-01', '2002-08-01',
               '2002-09-01', '2002-10-01',
               ...
               '2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01',
               '2024-05-01', '2024-06-01', '2024-07-01', '2024-08-01',
               '2024-09-01', '2024-10-01'],
              dtype='datetime64[ns]', length=274, freq=None)

In [15]:
# Flatten MultiIndex columns in stock_data
stock_data.columns = ['_'.join(col).strip() for col in stock_data.columns.values]

In [16]:
economic_data_filled.index.name = 'Date'
stock_data.index.name = 'Date'

# Create a daily date range based on stock data index
daily_date_range = pd.date_range(
    start=stock_data.index.min(),
    end=stock_data.index.max(),
    freq='D'  # Daily frequency
)

# Reindex economic data to daily frequency using forward fill
economic_data_daily = economic_data_filled.reindex(daily_date_range, method='ffill')

# Ensure the index name is consistent
economic_data_daily.index.name = 'Date'

# Merge the DataFrames using the date index
combined_data = stock_data.join(economic_data_daily, how='left')

# Check for missing values after merging
missing_values = combined_data.isna().sum()
print("Missing values after merging:")
print(missing_values)


Missing values after merging:
AMD_Open                       0
AMD_High                       0
AMD_Low                        0
AMD_Close                      0
AMD_Adj Close                  0
                              ..
Industrial Production Index    0
M2 Money Stock                 0
Crude Oil Prices               0
Retail Sales                   0
Total Business Inventories     0
Length: 313, dtype: int64


In [17]:
# Optionally, forward fill remaining missing values
combined_data.ffill(inplace=True)

In [18]:
# Inspect the head of the combined DataFrame
print(combined_data.head())

# Check the DataFrame info
combined_data.info()

            AMD_Open  AMD_High  AMD_Low  AMD_Close  AMD_Adj Close  AMD_Volume  \
Date                                                                            
2003-01-02      6.56      7.11     6.47       7.01           7.01    17474800   
2003-01-03      7.01      7.19     6.86       6.94           6.94     6235700   
2003-01-06      7.05      7.23     7.00       7.16           7.16     6833200   
2003-01-07      7.25      7.48     7.10       7.17           7.17     9785100   
2003-01-08      7.06      7.10     6.63       6.69           6.69    13952300   

            TSLA_Open  TSLA_High  TSLA_Low  TSLA_Close  ...  \
Date                                                    ...   
2003-01-02        NaN        NaN       NaN         NaN  ...   
2003-01-03        NaN        NaN       NaN         NaN  ...   
2003-01-06        NaN        NaN       NaN         NaN  ...   
2003-01-07        NaN        NaN       NaN         NaN  ...   
2003-01-08        NaN        NaN       NaN         NaN

In [22]:
print(combined_data.isna().any().sum())

72


In [23]:
combined_data.shape, stock_data.shape

((5514, 313), (5514, 300))

In [24]:
combined_data.head()

,AMD_Open,AMD_High,AMD_Low,AMD_Close,AMD_Adj Close,AMD_Volume,TSLA_Open,TSLA_High,TSLA_Low,TSLA_Close,...,Producer Price Index,Unemployment Rate,Nonfarm Payroll Employment,Real GDP,Housing Starts,Industrial Production Index,M2 Money Stock,Crude Oil Prices,Retail Sales,Total Business Inventories
Date,,,,,,,,,,,,,,,,,,,,,
2003-01-02,6.56,7.11,6.47,7.01,7.01,17474800,NaN,NaN,NaN,NaN,...,135.3,5.8,130580.0,14614.141,1853.0,91.1369,5812.3,29.581667,267230.0,1139589.0
2003-01-03,7.01,7.19,6.86,6.94,6.94,6235700,NaN,NaN,NaN,NaN,...,135.3,5.8,130580.0,14614.141,1853.0,91.1369,5812.3,29.581667,267230.0,1139589.0
2003-01-06,7.05,7.23,7.00,7.16,7.16,6833200,NaN,NaN,NaN,NaN,...,135.3,5.8,130580.0,14614.141,1853.0,91.1369,5812.3,29.581667,267230.0,1139589.0
2003-01-07,7.25,7.48,7.10,7.17,7.17,9785100,NaN,NaN,NaN,NaN,...,135.3,5.8,130580.0,14614.141,1853.0,91.1369,5812.3,29.581667,267230.0,1139589.0
2003-01-08,7.06,7.10,6.63,6.69,6.69,13952300,NaN,NaN,NaN,NaN,...,135.3,5.8,130580.0,14614.141,1853.0,91.1369,5812.3,29.581667,267230.0,1139589.0
